In [1]:
from ctapipe.core import Tool, Component
from traitlets import (Integer, Float, List, Dict,Unicode, TraitError, observe)
import logging
from time import sleep

see https://github.com/ipython/traitlets/blob/master/examples/myapp.py

# Setup:

Create a few `Component`s and put them into a `Tool`:

In [2]:
class MyComponent(Component):
    description = "Do some things"

    value = Integer(default_value=-1, help="Value to use").tag(config=True)

    def do_thing(self):
        self.log.debug("Did thing")

# in order to have 2 of the same components at once
class SecondaryMyComponent(MyComponent):
    pass


class AdvancedComponent(Component):
    name="AdvancedComponent"
    description = "something more advanced"

    value1 = Integer(default_value=-1, help="Value to use").tag(config=True)
    infile = Unicode(help="input file name").tag(config=True)
    outfile = Unicode(help="output file name").tag(config=True)

    @observe('outfile')
    def on_outfile_changed(self, change):
        self.log.warning("Outfile was changed to '{}'".format(change))


class MyTool(Tool):
    name="mytool"
    description="do some things and stuff"
    aliases = Dict(dict(infile='AdvancedComponent.infile',
                        iterations='MyTool.iterations'))

    # Which classes are registered for configuration
    classes = List([MyComponent, AdvancedComponent, SecondaryMyComponent])

    # local configuration parameters
    iterations = Integer(5,help="Number of times to run",allow_none=False).tag(config=True)

    def setup_comp(self):
        self.comp = MyComponent(self, config=self.config)
        self.comp2 = SecondaryMyComponent(self, config=self.config)

    def setup_advanced(self):
        self.advanced = AdvancedComponent(self, config=self.config)

    def setup(self):
        self.setup_comp()
        self.setup_advanced()

    def start(self):
        self.log.info("Performing {} iterations...".format(self.iterations))
        for ii in range(self.iterations):
            self.log.info("ITERATION {}".format(ii))
            self.comp.do_thing()
            self.comp2.do_thing()
            sleep(0.5)
            
    def finish(self):
        self.log.warning("Shutting down.")
    


# Get Help info

The following allows you to print the help info within a Jupyter notebook, but this same inforamtion would be displayed if the user types:
```
  mytool --help
```

In [3]:
tool=MyTool()

In [4]:
tool.print_help()

do some things and stuff

Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--log_level=<Enum> (Application.log_level)
    Default: 30
    Choices: (0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL')
    Set the log level by value or name.
--iterations=<Int> (MyTool.iterations)
    Default: 5
    Number of times to run
--config=<Unicode> (Tool.config_file)
    Default: ''
    name of a configuration file with parameters to load in addition to command-
    line parameters
--infile=<Unicode> (AdvancedComponent.infile)
    Default: ''
    input file name

To see all available configurables, use `--help-all`



The following  is equivalant to the user typing `mytool --help-all`

In [5]:
tool.print_help(classes=True)

do some things and stuff

Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--log_level=<Enum> (Application.log_level)
    Default: 30
    Choices: (0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL')
    Set the log level by value or name.
--iterations=<Int> (MyTool.iterations)
    Default: 5
    Number of times to run
--config=<Unicode> (Tool.config_file)
    Default: ''
    name of a configuration file with parameters to load in addition to command-
    line parameters
--infile=<Unicode> (AdvancedComponent.infile)
    Default: ''
    input file name

Class parameters
----------------

Parameters are set from command-line arguments of the form:
`--Class.trait=value`. This line is evaluated in Python, so simple expressions
are allowed, e.g.:: `--C.a='range(3)'` For setting C.a=[0,1,2].

MyTool options
-------------

# Run the tool

here we pass in argv since it is a Notebook, but if argv is not specified it's read from `sys.argv`, so the following is the same as running:

```sh
mytool --log_level=INFO --infile bork.txt --iterations=3
```

In [6]:
tool.log_format = "%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s" 
tool.run(argv=['--log_level','INFO','--infile','bork.txt','--iterations','3'])

2016-06-28 18:25:08 : INFO [MyTool initialize] version 0.0.dev663 [release=False] [githash=aae0d5150c2c8bd7957cbdf4f4b05cb74af506a7]
2016-06-28 18:25:08 : INFO [MyTool run] Starting: mytool
2016-06-28 18:25:08 : INFO [MyTool start] Performing 3 iterations...
2016-06-28 18:25:08 : INFO [MyTool start] ITERATION 0
2016-06-28 18:25:08 : INFO [MyTool start] ITERATION 1
2016-06-28 18:25:09 : INFO [MyTool start] ITERATION 2
2016-06-28 18:25:09 : WARNING [MyTool finish] Shutting down.


here we change the log-level to DEBUG:

In [7]:
tool.run(argv=['--log_level','DEBUG','--infile','bork.txt'])

2016-06-28 18:25:11 : INFO [MyTool initialize] version 0.0.dev663 [release=False] [githash=aae0d5150c2c8bd7957cbdf4f4b05cb74af506a7]
2016-06-28 18:25:11 : INFO [MyTool run] Starting: mytool
2016-06-28 18:25:11 : DEBUG [MyTool run] CONFIG: {'MyTool': {'log_level': 'DEBUG', 'iterations': 3}, 'AdvancedComponent': {'infile': 'bork.txt'}}
2016-06-28 18:25:11 : INFO [MyTool start] Performing 3 iterations...
2016-06-28 18:25:11 : INFO [MyTool start] ITERATION 0
2016-06-28 18:25:11 : DEBUG [MyTool.MyComponent do_thing] Did thing
2016-06-28 18:25:11 : DEBUG [MyTool.SecondaryMyComponent do_thing] Did thing
2016-06-28 18:25:12 : INFO [MyTool start] ITERATION 1
2016-06-28 18:25:12 : DEBUG [MyTool.MyComponent do_thing] Did thing
2016-06-28 18:25:12 : DEBUG [MyTool.SecondaryMyComponent do_thing] Did thing
2016-06-28 18:25:12 : INFO [MyTool start] ITERATION 2
2016-06-28 18:25:12 : DEBUG [MyTool.MyComponent do_thing] Did thing
2016-06-28 18:25:12 : DEBUG [MyTool.SecondaryMyComponent do_thing] Did thin

you can also set parameters directly in the class, rather than using the argument/configfile parser:

In [8]:
tool.iterations = 1
tool.log_level = 0
tool.run('')

2016-06-28 18:25:15 : WARNING [MyTool finish] Shutting down.


see what happens when a value is set that is not of the correct type:

In [9]:
try:
    tool.iterations = "badval"
except TraitError as E:
    print("bad value:",E)

bad value: The 'iterations' trait of a MyTool instance must be an int, but a value of 'badval' <class 'str'> was specified.


Example of what happens when you change a parameter that is being "observed" in a class. It's handler is called:

In [10]:
tool.advanced.outfile = "Another.txt"

2016-06-28 18:25:20 : WARNING [MyTool.AdvancedComponent on_outfile_changed] Outfile was changed to '{'new': 'Another.txt', 'name': 'outfile', 'old': '', 'type': 'change', 'owner': <__main__.AdvancedComponent object at 0x1040844e0>}'


we see that the handler for `outfile` was called, and it receive a change dict that shows the old and new values.

create a tool using a config file:

In [11]:
!cat Tools.json

{
    "MyTool": {"log_level":"DEBUG"},
    "AdvancedComponent": {"infile": "something.txt"}
}


In [12]:
tool2 = MyTool()

In [13]:
tool2.run(argv=['--config','Tools.json'])

 WARNING [MyTool]: Unrecognized JSON config file version, assuming version 1
    INFO [MyTool]: version 0.0.dev663 [release=False] [githash=aae0d5150c2c8bd7957cbdf4f4b05cb74af506a7]
    INFO [MyTool]: Starting: mytool
   DEBUG [MyTool]: CONFIG: {'MyTool': {'log_level': 'DEBUG', 'config_file': 'Tools.json'}, 'AdvancedComponent': {'infile': 'something.txt'}}
    INFO [MyTool]: Performing 5 iterations...
    INFO [MyTool]: ITERATION 0
   DEBUG [MyTool.MyComponent]: Did thing
   DEBUG [MyTool.SecondaryMyComponent]: Did thing
    INFO [MyTool]: ITERATION 1
   DEBUG [MyTool.MyComponent]: Did thing
   DEBUG [MyTool.SecondaryMyComponent]: Did thing
    INFO [MyTool]: ITERATION 2
   DEBUG [MyTool.MyComponent]: Did thing
   DEBUG [MyTool.SecondaryMyComponent]: Did thing
    INFO [MyTool]: ITERATION 3
   DEBUG [MyTool.MyComponent]: Did thing
   DEBUG [MyTool.SecondaryMyComponent]: Did thing
    INFO [MyTool]: ITERATION 4
   DEBUG [MyTool.MyComponent]: Did thing
   DEBUG [MyTool.SecondaryMyCompone

In [14]:
print(tool2.advanced.infile)

something.txt


In [15]:
print(tool2.config)

{'MyTool': {'log_level': 'DEBUG', 'config_file': 'Tools.json'}, 'AdvancedComponent': {'infile': 'something.txt'}}


In [16]:
tool2.initialized()
tool2.is_initialized

AttributeError: 'MyTool' object has no attribute 'is_initialized'

In [17]:
tool3 = MyTool()

In [18]:
tool3.is_initialized

AttributeError: 'MyTool' object has no attribute 'is_initialized'